In [84]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import time

In [85]:
df=pd.read_csv('modified_data.csv')

Drop_columns

In [86]:
df.drop('educational_num',axis=1,inplace=True)

In [87]:
df.head()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,24,Private,369667,Some-college,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K


In [88]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income'],
      dtype='object')

In [89]:
# Get all column names except 'income'
feature_columns = df.drop('income', axis=1).columns

# Select categorical columns
categorical_columns = df[feature_columns].select_dtypes(include=['object']).columns

# Select numerical columns
numerical_columns = df[feature_columns].select_dtypes(include=['int', 'float']).columns

In [90]:
categorical_columns

Index(['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'gender', 'native_country'],
      dtype='object')

#### Target Folder

In [91]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [92]:
df.head()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,25,Private,226802,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,24,Private,369667,Some-college,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K


In [93]:

categorical_columns

Index(['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'gender', 'native_country'],
      dtype='object')

In [94]:
numerical_columns

Index(['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week'], dtype='object')

Transformation

In [95]:
# Separate the target column
target = df['income']
features = df.drop('income', axis=1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [96]:
# Create the column transformer for numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('log_transform', FunctionTransformer(np.log1p, validate=False)),
            ('scaler', StandardScaler())
        ]), numerical_columns),
        ('categorical', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
        ]), categorical_columns)
    ])

### Transforming

In [97]:
# Fit and transform the features
transformed_X_train = preprocessor.fit_transform(X_train)
transformed_X_test = preprocessor.transform(X_test)

# Apply LabelEncoder to the target column
label_encoder = LabelEncoder()
transformed_y_train = label_encoder.fit_transform(y_train)
transformed_y_test = label_encoder.transform(y_test)

f:\Shivan_Sir_Internship\Project_1\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [98]:
# Create the KNN classifier
model = KNeighborsClassifier(n_neighbors=5)

# Train the model
model.fit(transformed_X_train, transformed_y_train)

# Make predictions on the test set
y_pred = model.predict(transformed_X_test)




In [99]:
# Calculate the accuracy score
accuracy = accuracy_score(transformed_y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8122263914946842


In [100]:

# Create a list of classification models
models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('KNN', KNeighborsClassifier()),
    ('SVM', SVC())
]

In [101]:
# Create an empty list to store results
results = []

In [102]:


# Iterate over the models and perform training and evaluation
for model_name, model in models:
    start_time = time.time()
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    end_time = time.time()
    
    # Calculate accuracy on test set
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate training time
    training_time = end_time - start_time
    
    # Append results to list
    results.append({'Model': model_name, 'Accuracy': accuracy, 'Training Time': training_time})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

# Print the results
print(results_df)

f:\Shivan_Sir_Internship\Project_1\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
f:\Shivan_Sir_Internship\Project_1\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
f:\Shivan_Sir_Internship\Project_1\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be 

                 Model  Accuracy  Training Time
0  Logistic Regression  0.835053       0.318534
1        Decision Tree  0.795184       0.405133
2        Random Forest  0.836617       2.611397
3                  KNN  0.812226       0.090402
4                  SVM  0.842714      18.574463
